In [3]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2021-11-02 09:37:40.209501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:37:40.218811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:37:40.219235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
name='j'
dataset='app_store'
encoding='sbert'
al_steps=100
sample_size_per_step=1
save_and_repeat=1
query_strategy='unc'
random=False

In [5]:
import sys, os
cwd = os.getcwd()
sys.path.append(cwd + '/../.') 

import math
import argparse
import tensorflow as tf
import numpy as np
from src.EvaluationLoop import EvaluationLoop
from src.training_loop2 import QueryStrategy
from data.datasets import Datasets, Encodings
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertTokenizerFast

In [6]:
    name_postfix = ""
    
    print('#############')
    print(random)
    print('#############')
    
    if random:
        print('Random query strategy is used. Selection is ignored.')
        name_postfix = "_rnd"
        query_strategy = QueryStrategy.unc
    
    print('load index ... ')
    used_training_index_dir = f'../eval_proxy/res/{dataset}_{encoding}_{str(query_strategy)}_{name}{name_postfix}_{sample_size_per_step}_{al_steps}_{save_and_repeat}/used_training_index.npy'
    
    print(used_training_index_dir)
    used_training_index = np.load(used_training_index_dir, allow_pickle=True)
    
    print('load data...')
    data_X = np.load(f'../data/datasets/{dataset}_X.npy')
    data_y = np.load(f'../data/datasets/{dataset}_y.npy')
    
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
    sss.get_n_splits(data_X, data_y)
    i = 0
    for train_index, test_index in sss.split(data_X, data_y):
        print(f'Model #{i} ...')
        X_train, X_test = data_X[train_index], data_X[test_index]
        y_train, y_test = data_y[train_index], data_y[test_index]
        
        idx = used_training_index[i][-1]
        
        X_used = X_train[idx]
        y_used = y_train[idx]
        
        X_bert_train, X_bert_eval, y_bert_train, y_bert_eval = train_test_split(X_used, y_used, test_size=0.10, random_state=42, stratify=y_used)
        
        del X_used
        del y_used
        del X_train
        del y_train
        
        print('tokenize ...')
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
        train_encodings = tokenizer(X_bert_train.tolist(), truncation=True, padding=True)
        val_encodings = tokenizer(X_bert_eval.tolist(), truncation=True, padding=True)
        test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)
        
        del X_bert_train
        del X_bert_eval
        del X_test
        del tokenizer

        train_dataset = tf.data.Dataset.from_tensor_slices((
            dict(train_encodings),
            y_bert_train
        ))
        val_dataset = tf.data.Dataset.from_tensor_slices((
            dict(val_encodings),
            y_bert_eval
        ))
        #test_dataset = tf.data.Dataset.from_tensor_slices((
        #    dict(test_encodings),
        #    y_test
        #))
        
        del train_encodings
        del val_encodings
        del test_encodings
        
        break;

#############
False
#############
load index ... 
../eval_proxy/res/app_store_sbert_unc_j_1_100_1/used_training_index.npy
load data...
Model #0 ...
tokenize ...


2021-11-02 09:37:56.207272: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-02 09:37:56.208509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:37:56.208986: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:37:56.209366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [7]:
        training_args = TFTrainingArguments(
                    evaluation_strategy= "steps",
                    output_dir='./bert_models',  
                    num_train_epochs=10,              # total number of training epochs
                    per_device_train_batch_size=16,  # batch size per device during training
                    per_device_eval_batch_size=32,   # batch size for evaluation
                    weight_decay=0.01,               # strength of weight decay
                    warmup_steps=math.ceil(len(y_bert_train) / 16),
                    logging_dir='./logs',         # directory for storing logs
                    logging_steps=2000,
                    learning_rate= 5e-5,
                    save_total_limit = 1,
                    #load_best_model_at_end= True,
                )
        
        print('Load BERT model ...')
        
        with training_args.strategy.scope():
            model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

        trainer = TFTrainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset,         # training dataset
            eval_dataset=val_dataset             # evaluation dataset
        )
        

Load BERT model ...


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.9/site-packages/transformers/trainer_tf.py:109: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  warnings.warn(


In [ ]:
1

In [ ]:
        training_args = TFTrainingArguments(
                    evaluation_strategy= "steps",
                    output_dir='./bert_models',  
                    num_train_epochs=10,              # total number of training epochs
                    per_device_train_batch_size=16,  # batch size per device during training
                    per_device_eval_batch_size=32,   # batch size for evaluation
                    #warmup_steps=500,                # number of warmup steps for learning rate scheduler
                    weight_decay=0.01,               # strength of weight decay
                    warmup_steps=math.ceil(len(y_bert_train) / 16),
                    logging_dir='./logs',         # directory for storing logs
                    logging_steps=2000,
                    learning_rate= 5e-5,
                    save_total_limit = 1,
                    #load_best_model_at_end= True,
                )
        
        print('Load BERT model ...')
        
        with training_args.strategy.scope():
            model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

        trainer = TFTrainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset,         # training dataset
            eval_dataset=val_dataset             # evaluation dataset
        )
        
        print('Train BERT model ...')
        trainer.train()
        
        print('Predict ...')
        preds = trainer.model.predict(test_dataset.batch(8))
        p_softmax = tf.nn.softmax(preds.logits, axis=1).numpy()
        embeddings = sbert.encode(X_used)
        X_test = sbert.encode(X_test)
        y_pred = p_softmax.argmax(axis=1)

        np.save(f'./res/{dataset}_{encoding}_{str(query_strategy)}_{name}{name_postfix}_{sample_size_per_step}_{al_steps}_{save_and_repeat}/y_pred_{i}', y_pred)
        np.save(f'./res/{dataset}_{encoding}_{str(query_strategy)}_{name}{name_postfix}_{sample_size_per_step}_{al_steps}_{save_and_repeat}/y_true_{i}', y_test)
        
        from sklearn.metrics import f1_score
        print(f1_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='macro'))

        i = i + 1

In [1]:
import sys, os
cwd = os.getcwd()
sys.path.append(cwd + '/../.') 

from src.training_loop2 import QueryStrategy
from data.datasets import Datasets, Encodings

In [2]:
import math
import argparse
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertTokenizerFast

In [3]:
name='j'
dataset='app_store'
encoding='sbert'
al_steps=100
sample_size_per_step=1
save_and_repeat=1
query_strategy='unc'
random=False

In [4]:
name_postfix = ""
if random:
    print('Random query strategy is used. Selection is ignored.')
    name_postfix = "_rnd"
    query_strategy = QueryStrategy.unc

print('load index ... ')
used_training_index_dir = f'../eval_proxy/res/{dataset}_{encoding}_{str(query_strategy)}_{name}{name_postfix}_{sample_size_per_step}_{al_steps}_{save_and_repeat}/used_training_index.npy'

load index ... 


In [5]:
print(used_training_index_dir)
used_training_index = np.load(used_training_index_dir, allow_pickle=True)

../eval_proxy/res/app_store_sbert_unc_j_1_100_1/used_training_index.npy


In [16]:
print('load data...')
data_X = np.load(f'../data/datasets/{dataset}_X.npy')
data_y = np.load(f'../data/datasets/{dataset}_y.npy')

load data...


In [17]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
sss.get_n_splits(data_X, data_y)

5

In [18]:
i = 0
for train_index, test_index in sss.split(data_X, data_y):
    print(f'Model #{i} ...')
    X_train, X_test = data_X[train_index], data_X[test_index]
    y_train, y_test = data_y[train_index], data_y[test_index]
        
    idx = used_training_index[i][-1]
        
    X_used = X_train[idx]
    y_used = y_train[idx]
    break;
        

Model #0 ...


In [19]:
X_bert_train, X_bert_eval, y_bert_train, y_bert_eval = train_test_split(X_used, y_used, test_size=0.10, random_state=42, stratify=y_used)

In [20]:
print('tokenize ...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenize ...


In [21]:
train_encodings = tokenizer(X_bert_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_bert_eval.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

In [22]:
X_bert_train = None
X_bert_eval = None
X_test = None
tokenizer = None

In [23]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_bert_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_bert_eval
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

2021-11-02 09:18:05.085959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:18:05.096358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:18:05.096864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 09:18:05.097995: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [24]:
train_encodings=None
val_encodings=None
test_encodings=None